# 象型柱状图
## 请选择年份
年份的可选值： 2010, 2011, 2012, 2014, 2016, 2018

In [1]:
year = 2010

In [2]:
print([table for table in cfps[year]])

['adult', 'child', 'comm', 'famconf', 'famecon']


In [3]:
table = 'adult'

In [4]:
schema = cfps[year][table].schema

## 请选择一列数据
变量与您选择的table和year有关

In [5]:
print("可选择的变量：")
i = 0
for var, value in schema.items():
    if value['type'] in ['enum','int32','float64'] \
    and '选择' not in value['key'] :
        print("{}:{}".format(var,value['key']))
        i += 1
print("*[一共有{}个变量]".format(i))

可选择的变量：
pid:个人id
fid:家庭样本代码
cid:村居样本代码
provcd:省国标码
countyid:区县顺序码
psu:primary sampling unit
rswt_nat:个人权重-全国样本
rswt_res:个人权重-全国再抽样样本
subpopulation:抽样子总体
subsample:是否在全国再抽样样本中
indno:个人的家庭成员顺序号
urban:基于国家统计局资料的城乡分类变量
cyear:采访年份
cmonth:访问月份
gender:性别（家庭问卷tb2）
qa1y:请问您的出生日期（年）
qa1y_best:您的出生日期修正（年）
qa1m:请问您的出生日期（月）
qa1age:自动计算年龄
qa101:请问您出生时的体重（斤）
qa102acode:请问您的出生地（省编码）
qa102c_code:请问您的出生地（县/区）_编码
qa2:您现在的户口状况是
qa201acode:您现在的户口落在什么地方（省编码）-清理
qa3:您3岁时的居住地是否与出生地相同
qa301acode:您3岁时的居住地在什么地方（省）
qa301c_code:3岁时居住地（县/区）_编码
qa302:您3岁时户口状况是
qa303:您3岁以前您父亲不与您在一起居住的连续时间（周）
qa304:您3岁以前您母亲不与您在一起居住的连续时间（周）
qa4:您12岁时的居住地是否与出生地相同
qa401acode:您12岁时的居住地在什么地方（省编码）
qa401c_code:12岁时居住地（县/区）_编码
qa402:您12岁时户口状况是
qa403:您4-12岁时，您父亲不与您在一起居住的连续时间（周）
qa404:您4-12岁时，您母亲不与您在一起居住的连续时间（周）
qa5code:请问，您的民族成分（编码）
qa701:您是哪年入党的
qb1:请问您一共有几个兄弟姐妹（包含已经去世的）
qb2:有几个兄弟姐妹住在这个家里
qb3count:不住在一起兄弟姐妹人数（自动计数）
qb301_a_1:姓名1：家庭关系
qb302_a_1:姓名1：血缘关系
qb303y_a_1:姓名1：出生日期（年）
qb303m_a_1:姓名1：出生日期（月）
qb303a_a_1:姓名1：属相
qb303b_a_1:姓名1：年龄
qb304_a_1

In [6]:
var = 'income'

In [7]:
info = cfps[year][table].schema[var]
info

{'type': 'int32',
 'key': '个人收入',
 'range': {'min': -8, 'max': 800000},
 'details': ['income 取值多于 500 个，已转换 range 为最大、最小值'],
 'minmax': [-8.0, 800000.0]}

In [8]:
condi = f'{var}>0'

In [9]:
data = sql(f'select {var} from {table}_{year} where {condi}')
counts = data.value_counts()
minmax = info['minmax']

# 分成五个区间，可自行决定
grid = 5
minmax[0] = 0 if minmax[0]<0 else minmax[0]
instance = (minmax[-1]-minmax[0])/grid
area = [int(minmax[0]+instance*i) for i in range(grid+1)]
area
# instance

[0, 160000, 320000, 480000, 640000, 800000]

In [10]:

title1 = info['key']
condi1 = f'{var}>0'
data1 = sql(f'select {var} from {table}_{year} where {condi}')
counts = data.value_counts().sort_index()
x1 = []
y1 = []
m = 0
 
for a,b in zip(area[:-1],area[1:]):
    x1.append(f'[{a},{b})')
    for i,n in counts.items():
        if a<=i[0]<b:             
            m += n
    y1.append(m)
    n = 0

## 可视化
### 数值型

In [11]:
x1,y1

(['[0,160000)',
  '[160000,320000)',
  '[320000,480000)',
  '[480000,640000)',
  '[640000,800000)'],
 [24992, 25040, 25043, 25047, 25048])

In [12]:
(
    PictorialBar()
    .add_xaxis(x1)
    .add_yaxis(
        "",
        y1,
        label_opts=opts.LabelOpts(is_show=False),
        symbol_size=17,
        symbol_repeat="fixed",
        symbol_offset=[0, 0],
        is_symbol_clip=True,
        symbol=SymbolType.ROUND_RECT,
    )
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title=title1),
        xaxis_opts=opts.AxisOpts(is_show=False),
        yaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_show=False),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(opacity=0)
            ),
        ),
    )
    .render_notebook()
)

### 非数值型

In [13]:
var = 'mparty'

In [14]:
info = cfps[year][table].schema[var]
info

{'type': 'enum',
 'key': '母亲政治面貌',
 'range': {'-10': '无法判断',
  '-9': '缺失',
  '-8': '不适用',
  '-2': '拒绝回答',
  '-1': '不知道',
  '1': '共产党员',
  '2': '民主党派',
  '3': '共青团员',
  '4': '群众'},
 'details': [],
 'minmax': [1.0, 4.0]}

In [15]:
title = info['key']
data = sql(f'select {var} from {table}_{year} where {condi}')
# 统计每个x值的个数
counts = data.value_counts()
# counts
# print(counts)
y = []
x = []
for i,n in counts.items():
    # print(str(inti[0]))
    y.append(n)
    x.append(info['range'][str(int(i[0]))])
x,y

(['群众', '共产党员', '共青团员', '民主党派'], [19418, 424, 411, 11])

In [16]:
 (
    PictorialBar()
    .add_xaxis(x)
    .add_yaxis(
        "",
        y,
        label_opts=opts.LabelOpts(is_show=False),
        symbol_size=18,
        symbol_repeat="fixed",
        symbol_offset=[0, 0],
        is_symbol_clip=True,
        symbol=SymbolType.ROUND_RECT,
    )
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title=title),
        xaxis_opts=opts.AxisOpts(is_show=False),
        yaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_show=False),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(opacity=0)
            ),
        ),
    )
    .render_notebook()
)